<a href="https://colab.research.google.com/github/jeantirole/repo001/blob/master/0720_Jeju_append_ver04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
ROOT = '/content/drive'
drive.mount(ROOT)

import pandas as pd
from pandas.io import gbq

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action = 'ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from os.path import join  

MY_GOOGLE_DRIVE_PATH = 'My Drive/Data_Corona'
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)
print(PROJECT_PATH)

/content/drive/My Drive/Data_Corona


In [4]:
%cd "{PROJECT_PATH}"
!ls

/content/drive/My Drive/Data_Corona
Case.csv	  Policy.csv	   SeoulFloating.csv  TimeGender.csv
PatientInfo.csv   Region.csv	   TimeAge.csv	      TimeProvince.csv
PatientRoute.csv  SearchTrend.csv  Time.csv	      Weather.csv


In [5]:
P001= pd.read_csv('/content/drive/My Drive/Data_Corona/PatientInfo.csv')

In [6]:
import time 

In [7]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [8]:
# train load from v 
%%time

from google.cloud import bigquery
from tabulate import tabulate
import pandas as pd

project_id = 'eric-dacon'
client = bigquery.Client(project=project_id)

train = client.query('''
  SELECT 
    * 
  FROM `eric-dacon.jeju_data_ver1.201901_202003_train` 
  LIMIT 200000
  ''').to_dataframe()

CPU times: user 7.24 s, sys: 1.14 s, total: 8.38 s
Wall time: 26.1 s


In [9]:
# submission load from v 

%%time
project_id = 'eric-dacon'
client = bigquery.Client(project=project_id)

submission = client.query('''
  SELECT 
    * 
  FROM `eric-dacon.jeju_data_ver1.submission` 
  LIMIT 200000
  ''').to_dataframe()

CPU times: user 53.3 ms, sys: 1 ms, total: 54.3 ms
Wall time: 1.83 s


In [10]:
whos

Variable               Type         Data/Info
---------------------------------------------
MY_GOOGLE_DRIVE_PATH   str          My Drive/Data_Corona
P001                   DataFrame          patient_id     sex <...>n[5165 rows x 14 columns]
PROJECT_PATH           str          /content/drive/My Drive/Data_Corona
ROOT                   str          /content/drive
auth                   module       <module 'google.colab.aut<...>es/google/colab/auth.py'>
bigquery               module       <module 'google.cloud.big<...>ud/bigquery/__init__.py'>
client                 Client       <google.cloud.bigquery.cl<...>object at 0x7f783eac1a20>
drive                  module       <module 'google.colab.dri<...>s/google/colab/drive.py'>
gbq                    module       <module 'pandas.io.gbq' f<...>ckages/pandas/io/gbq.py'>
join                   function     <function join at 0x7f786c13ff28>
mpl                    module       <module 'matplotlib' from<...>/matplotlib/__init__.py'>
pd            

In [11]:
P001

,patient_id,sex,age,country,province,city,infection_case,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,male,50s,Korea,Seoul,Gangseo-gu,overseas inflow,NaN,75,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,male,30s,Korea,Seoul,Jungnang-gu,overseas inflow,NaN,31,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,male,50s,Korea,Seoul,Jongno-gu,contact with patient,2002000001,17,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,male,20s,Korea,Seoul,Mapo-gu,overseas inflow,NaN,9,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,female,20s,Korea,Seoul,Seongbuk-gu,contact with patient,1000000002,2,NaN,2020-01-31,2020-02-24,NaN,released
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5160,7000000015,female,30s,Korea,Jeju-do,Jeju-do,overseas inflow,NaN,25,NaN,2020-05-30,2020-06-13,NaN,released
5161,7000000016,NaN,NaN,Korea,Jeju-do,Jeju-do,overseas inflow,NaN,NaN,NaN,2020-06-16,2020-06-24,NaN,released
5162,7000000017,NaN,NaN,Bangladesh,Jeju-do,Jeju-do,overseas inflow,NaN,72,NaN,2020-06-18,NaN,NaN,isolated
5163,7000000018,NaN,NaN,Bangladesh,Jeju-do,Jeju-do,overseas inflow,NaN,NaN,NaN,2020-06-18,NaN,NaN,isolated


In [ ]:
# backup  

for i in p001: 
  A001 = P001[P001['province'] == i]
  a002 = A001['confirmed_date'].value_counts()
  a002 = pd.DataFrame(a002).reset_index()
  a002 = a002.rename({'index':'confirmed_date','confirmed_date':'total_patient'},axis='columns')
  #datetime 형태로 전환하기 
  a002['confirmed_date'] = pd.to_datetime(a002.confirmed_date, errors='coerce')
  a002['month_date'] = a002['confirmed_date'].dt.strftime('%m')
  a002['year_date'] = a002['confirmed_date'].dt.strftime('%Y')

  r001 = a002.groupby(['year_date','month_date'])['total_patient'].sum()
  r001 = pd.DataFrame(r001)
  r001 = r001.reset_index()
  r001['CARD_SIDO_NM'] = i

  result1 = pd.concat([r001])


  print(r001)

  result1 = pd.concat([df1,df2])
print(result1)


In [16]:
result = {'year_date':[0], 
        'month_date':[0],
        'total_patient':[0],
        'CARD_SIDO_NM':[0]}
result = pd.DataFrame(result)

p001 = list(P001['province'].unique())

for i in p001: 
  A001 = P001[P001['province'] == i]
  a002 = A001['confirmed_date'].value_counts()
  a002 = pd.DataFrame(a002).reset_index()
  a002 = a002.rename({'index':'confirmed_date','confirmed_date':'total_patient'},axis='columns')
  #datetime 형태로 전환하기 
  a002['confirmed_date'] = pd.to_datetime(a002.confirmed_date, errors='coerce')
  a002['month_date'] = a002['confirmed_date'].dt.strftime('%m')
  a002['year_date'] = a002['confirmed_date'].dt.strftime('%Y')

  r001 = a002.groupby(['year_date','month_date'])['total_patient'].sum()
  r001 = pd.DataFrame(r001)
  r001 = r001.reset_index()
  r001['CARD_SIDO_NM'] = i
  result = result.append(r001)



In [24]:
result[result['CARD_SIDO_NM']=='Gyeonggi-do']

,year_date,month_date,total_patient,CARD_SIDO_NM
0,2020,01,2,Gyeonggi-do
1,2020,02,82,Gyeonggi-do
2,2020,03,417,Gyeonggi-do
3,2020,04,177,Gyeonggi-do
4,2020,05,175,Gyeonggi-do
5,2020,06,355,Gyeonggi-do


In [22]:
result['CARD_SIDO_NM'].unique()

array([0, 'Seoul', 'Busan', 'Daegu', 'Gwangju', 'Incheon', 'Daejeon',
       'Gyeonggi-do', 'Sejong', 'Ulsan', 'Gangwon-do',
       'Chungcheongbuk-do', 'Chungcheongnam-do', 'Jeollabuk-do',
       'Jeollanam-do', 'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Jeju-do'],
      dtype=object)

In [17]:
#17개 시군구에 대한, 년,월별 코로나 환자 숫자 합계
result
#result 데이터 안에 있는, CARD_SIDO_NM 들, 한글로 변환 

,year_date,month_date,total_patient,CARD_SIDO_NM
0,0,0,0,0
0,2020,01,7,Seoul
1,2020,02,79,Seoul
2,2020,03,392,Seoul
3,2020,04,156,Seoul
...,...,...,...,...
0,2020,02,2,Jeju-do
1,2020,03,7,Jeju-do
2,2020,04,4,Jeju-do
3,2020,05,2,Jeju-do


In [16]:
train.CARD_SIDO_NM.unique()

array(['강원', '서울', '경기', '제주', '전북', '인천', '부산', '경남', '충북', '대전', '충남',
       '경북', '광주', '대구', '울산', '전남', '세종'], dtype=object)

In [25]:
result01 = result['CARD_SIDO_NM'].replace({'Seoul':'서울',
                        'Busan':'부산',
                        'Daegu':'대구',
                        'Gwangju':'광주',
                        'Incheon':'인천',
                        'Daejeon':'대전',
                        'Gyeonggi-do':'경기',
                        'Sejong':'세종', 
                        'Ulsan':'울산', 
                        'Gangwon-do':'강원',
                        'Chungcheongbuk-do':'충북',
                        'Chungcheongnam-do':'충남', 
                        'Jeollabuk-do':'전북',
                        'Jeollanam-do':'전남',
                        'Gyeongsangbuk-do':'경북', 
                        'Gyeongsangnam-do':'경남', 
                        'Jeju-do':'제주'}, inplace=True)

In [30]:
result[result.CARD_SIDO_NM=='경기']

,year_date,month_date,total_patient,CARD_SIDO_NM
0,2020,01,2,경기
1,2020,02,82,경기
2,2020,03,417,경기
3,2020,04,177,경기
4,2020,05,175,경기
5,2020,06,355,경기


In [31]:
result

,year_date,month_date,total_patient,CARD_SIDO_NM
0,0,0,0,0
0,2020,01,7,서울
1,2020,02,79,서울
2,2020,03,392,서울
3,2020,04,156,서울
...,...,...,...,...
0,2020,02,2,제주
1,2020,03,7,제주
2,2020,04,4,제주
3,2020,05,2,제주


In [32]:
result01 = result[result['CARD_SIDO_NM'] != 0]
result01

,year_date,month_date,total_patient,CARD_SIDO_NM
0,2020,01,7,서울
1,2020,02,79,서울
2,2020,03,392,서울
3,2020,04,156,서울
4,2020,05,228,서울
...,...,...,...,...
0,2020,02,2,제주
1,2020,03,7,제주
2,2020,04,4,제주
3,2020,05,2,제주


In [21]:
# 02) Train 데이터 수정하기 
# datetime 형태로 전환하기 

In [40]:
train01 = train.copy()

In [41]:
train01['REG_YYMM'] = pd.to_datetime(train01.REG_YYMM, format = '%Y%m', errors = 'coerce')
train01['month_date'] = train01['REG_YYMM'].dt.strftime('%m')
train01['year_date'] = train01['REG_YYMM'].dt.strftime('%Y')

In [42]:
# 새로 만들어진 month_date 와 year_date 형태 컬럼 확인하기 
train01

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,month_date,year_date
0,2019-01-01,강원,화천군,화장품 및 방향제 소매업,강원,화천군,10s,2,1,4,30150,5,01,2019
1,2019-05-01,강원,평창군,기타음식료품위주종합소매업,강원,평창군,10s,2,1,7,94310,26,05,2019
2,2019-07-01,강원,양양군,체인화 편의점,강원,양양군,10s,2,1,51,342300,120,07,2019
3,2019-07-01,서울,동작구,택시 운송업,강원,정선군,10s,1,1,8,111500,19,07,2019
4,2019-02-01,서울,중구,택시 운송업,강원,속초시,10s,1,1,45,558670,153,02,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,2020-01-01,경기,가평군,기타 외국식 음식점업,경기,가평군,20s,1,2,4,124500,4,01,2020
199996,2020-01-01,경기,가평군,차량용 주유소 운영업,경기,용인시 수지구,20s,2,1,3,192000,3,01,2020
199997,2020-01-01,경기,고양시 덕양구,비알콜 음료점업,경기,이천시,20s,2,1,4,26200,6,01,2020
199998,2020-01-01,경기,고양시 덕양구,중식 음식점업,경기,고양시 일산동구,20s,2,2,6,301500,7,01,2020


In [43]:
len(train01.CARD_SIDO_NM.unique())

17

In [44]:
train01.year_date.unique()

array(['2019', '2020'], dtype=object)

In [45]:
result01.year_date.unique()

array(['2020'], dtype=object)

In [48]:
train01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   REG_YYMM      200000 non-null  datetime64[ns]
 1   CARD_SIDO_NM  200000 non-null  object        
 2   CARD_CCG_NM   199579 non-null  object        
 3   STD_CLSS_NM   200000 non-null  object        
 4   HOM_SIDO_NM   200000 non-null  object        
 5   HOM_CCG_NM    200000 non-null  object        
 6   AGE           200000 non-null  object        
 7   SEX_CTGO_CD   200000 non-null  int64         
 8   FLC           200000 non-null  int64         
 9   CSTMR_CNT     200000 non-null  int64         
 10  AMT           200000 non-null  int64         
 11  CNT           200000 non-null  int64         
 12  month_date    200000 non-null  object        
 13  year_date     200000 non-null  object        
dtypes: datetime64[ns](1), int64(5), object(8)
memory usage: 21.4+ MB


In [49]:
result01.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89 entries, 0 to 4
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year_date      89 non-null     object
 1   month_date     89 non-null     object
 2   total_patient  89 non-null     int64 
 3   CARD_SIDO_NM   89 non-null     object
dtypes: int64(1), object(3)
memory usage: 3.5+ KB


In [50]:
result01[result01['CARD_SIDO_NM']=='경기']

,year_date,month_date,total_patient,CARD_SIDO_NM
0,2020,01,2,경기
1,2020,02,82,경기
2,2020,03,417,경기
3,2020,04,177,경기
4,2020,05,175,경기
5,2020,06,355,경기


In [51]:
C001 = pd.merge(train01, result01, how='left')
C001

# 겹치는게 나오는 이유 ,, 1) index 문제다. 2) data type 문제다 

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,month_date,year_date,total_patient
0,2019-01-01,강원,화천군,화장품 및 방향제 소매업,강원,화천군,10s,2,1,4,30150,5,01,2019,NaN
1,2019-05-01,강원,평창군,기타음식료품위주종합소매업,강원,평창군,10s,2,1,7,94310,26,05,2019,NaN
2,2019-07-01,강원,양양군,체인화 편의점,강원,양양군,10s,2,1,51,342300,120,07,2019,NaN
3,2019-07-01,서울,동작구,택시 운송업,강원,정선군,10s,1,1,8,111500,19,07,2019,NaN
4,2019-02-01,서울,중구,택시 운송업,강원,속초시,10s,1,1,45,558670,153,02,2019,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,2020-01-01,경기,가평군,기타 외국식 음식점업,경기,가평군,20s,1,2,4,124500,4,01,2020,2.0
199996,2020-01-01,경기,가평군,차량용 주유소 운영업,경기,용인시 수지구,20s,2,1,3,192000,3,01,2020,2.0
199997,2020-01-01,경기,고양시 덕양구,비알콜 음료점업,경기,이천시,20s,2,1,4,26200,6,01,2020,2.0
199998,2020-01-01,경기,고양시 덕양구,중식 음식점업,경기,고양시 일산동구,20s,2,2,6,301500,7,01,2020,2.0
